In [2]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

examples = [
    { "movie": "Top Gun", "answer": "🛩️👨‍✈️🔥" },
    { "movie": "The Godfather", "answer": " 👨‍👨‍👦🔫🍝" },
    { "movie": "ET", "answer": "👽🚲🌕" }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Give me 3 emojis that represent the movie {movie}"),
    ("ai", "{answer}")
])

fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt, 
    examples = examples
)

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.1,
    streaming=True, 
    callbacks=[StreamingStdOutCallbackHandler()]
)

memory = ConversationSummaryMemory(
    llm=llm, max_token_limit=80,
    memory_key="history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", 
        "You are a movie expert who always responds with 3 emojis representing the movie "
    ),
    fewshot_prompt,
    MessagesPlaceholder(variable_name="history"),  # Keeps conversation memory
    ("human", "Answer the question. If question is the word of movie, then Give me 3 emojis that represent the movie {movie}")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt| llm

# Function to invoke the chain and store results in memory
def invoke_chain(movie):
    response = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie}, 
        {"output": response.content}
    )  # Save conversation history
    print(response)

/var/folders/dd/dvf_l6wd65n98d8w8p8_w6_w0000gn/T/ipykernel_94861/3562914895.py:24: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/var/folders/dd/dvf_l6wd65n98d8w8p8_w6_w0000gn/T/ipykernel_94861/3562914895.py:31: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [3]:
invoke_chain("Interstellar")

🚀⏳🌌The human mentions "Interstellar" and the AI responds with rocket emoji, hourglass emoji, and galaxy emoji.content='🚀⏳🌌' additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run-e13d8717-9a35-4711-bf77-cfe07fdf06f9-0'


In [4]:
invoke_chain("The Matrix")

🕶️💊🔵The human mentions "Interstellar" and the AI responds with rocket emoji, hourglass emoji, and galaxy emoji. When the human brings up "The Matrix," the AI responds with sunglasses emoji, pill emoji, and blue circle emoji.content='🕶️💊🔵' additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run-290ba74d-c257-4fe3-ab08-b96471613260-0'


In [5]:
invoke_chain("What is the movie that I asked first time?")

🚀⏳🌌The human mentions "Interstellar" and the AI responds with rocket emoji, hourglass emoji, and galaxy emoji. When the human brings up "The Matrix," the AI responds with sunglasses emoji, pill emoji, and blue circle emoji. The human asks about the movie mentioned first, and the AI responds with 🚀⏳🌌.content='🚀⏳🌌' additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run-e571cc78-86d8-46d4-a358-94db429d2202-0'
